In [13]:
from dotenv import load_dotenv
import os, requests, sys, json
from IPython.display import JSON
from pathlib import Path
import logging.handlers
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.signal import convolve2d
import warnings
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Helpers.Exception_Handling import Exception_Handling as eh
from Helpers.Mongo_Interface import Mongo_Interface as mi

env_path = Path(fr"{os.environ['USERPROFILE']}\.env") 
load_dotenv(dotenv_path=env_path)

True

In [14]:
results = mi.get_all()
len(results)

39

In [15]:
horizontal_kernel = np.array([[ 1, 1, 1, 1]])
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(4, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

def winning_move(board, player):
    for kernel in detection_kernels:
        if (convolve2d(board == player, kernel, mode="valid") == 4).any():
            return True
    return False

In [16]:
columns = [item for item in range(0, 42)]
columns.append('winner')
columns.append('looser')
columns.append('is_stale')

df = pd.DataFrame(columns=columns)

for i in range(len(results)):
    flat_board = results[i]['steps'][-1:][0][0]['observation']['board'].copy()
    board = np.resize(flat_board, (6,7))
    one_winning_move = winning_move(board, 1)
    if winning_move(board, 1):
        flat_board.append(1)
        flat_board.append(2)
        flat_board.append(False)
    elif winning_move(board, 2):
        flat_board.append(2)
        flat_board.append(1)
        flat_board.append(False)
    else:
        flat_board.append(None)
        flat_board.append(None)
        flat_board.append(True)
    df2 = pd.DataFrame(data=[flat_board], columns=columns)
    df = pd.concat([df, df2])
df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,winner,looser,is_stale
0,1,0,0,0,1,0,0,1,2,0,...,1,1,2,2,1,1,2,1,2,False
0,0,0,0,2,0,0,0,0,0,0,...,1,1,2,2,1,1,1,1,2,False
0,0,0,2,0,0,0,0,2,0,1,...,1,0,2,1,2,2,0,1,2,False
0,0,0,0,0,2,0,0,0,0,0,...,1,1,1,2,2,1,1,1,2,False
0,2,0,0,0,0,0,0,2,0,0,...,2,1,1,2,1,0,0,1,2,False
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,2,2,2,2,1,False
0,0,0,2,0,0,0,0,0,0,1,...,1,1,2,1,2,0,2,1,2,False
0,0,0,0,0,0,0,0,0,0,1,...,1,1,1,2,1,2,1,2,1,False
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,2,1,1,2,2,1,False
0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,1,1,2,1,2,False


In [93]:
JSON(results[16]['steps'][-1:])

<IPython.core.display.JSON object>

In [ ]:
for i in range(len(results)):
    flat_board = results[i]['steps'][-1:][0][0]['observation']['board']
    board = np.resize(flat_board, (6,7))
    print(board)
    one_winning_move = winning_move(board, 1)
    if one_winning_move:
        flat_board.append(1)
        flat_board.append(2)
    else:
        flat_board.append(2)
        flat_board.append(1)
    print(flat_board)
    df2 = pd.DataFrame(data=flat_board, columns=columns)
    df = df.append(df2, ignore_index=True)

In [50]:
df

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,winner,looser


In [54]:
len(flat_board)

54

In [55]:
columns

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 'winner',
 'looser']

In [4]:
json.dumps({ "steps" : { "$elemMatch" : { "0.observation.kore" : { "$exists": True } } } })

'{"steps": {"$elemMatch": {"0.observation.kore": {"$exists": true}}}}'

In [15]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import json
import os

mongo_host = 'mongodb+srv://cluster0.bp0a9tr.mongodb.net/?retryWrites=true&w=majority'
mongo_authMechanism= 'SCRAM-SHA-256'

db_name = 'kaggle_connectx'
collection_name = 'history'

def initialize_db():
    client = MongoClient(mongo_host,
                            username=os.getenv("kaggleconnectx_username"),
                            password=os.getenv("kaggleconnectx_password"))
    db = client[db_name]
    collection = db[collection_name]

    return collection

def insert_payload(payload):
    collection = initialize_db()
    insert_id = collection.insert_one(payload).inserted_id
    return insert_id

def empty_collection():
    collection = initialize_db()
    collection.delete_many({})
    
def get_all():
    results = []
    collection = initialize_db()
    doc_cur = collection.find({ "steps" : { "$elemMatch" : { "0.observation.kore" : { "$exists": False } } } })
   
    for doc in doc_cur:

        results.append(doc)
    return results


In [ ]:
result = get_all()

In [14]:
len(result)

39